# Extract Tweets

Using keywords found in original, reply or retweets

In [16]:
import pandas as pd
import json
import tweepy
import pickle
from datetime import datetime
import os

In [44]:
# Paths for data
homepath = '../../'
covid_path = homepath + 'covid_data/'
csv_path = covid_path + 'reparsed_csvs/'
save_path = covid_path + 'retweet_jsons/'

In [36]:
orig_dfs,qtd_dfs,rt_dfs = [],[],[]
for i in os.listdir(csv_path):
    if 'orig_' in i:
        orig_dfs.append(i)
    elif 'qtd_' in i:
        qtd_dfs.append(i)
    elif 'rt_' in i:
        rt_dfs.append(i)
        
orig_dfs = sorted(orig_dfs)
qtd_dfs = sorted(qtd_dfs)
rt_dfs = sorted(rt_dfs)

orig_consp = pd.DataFrame()
qtd_consp = pd.DataFrame()
rt_consp = pd.DataFrame()
for i in range(len(orig_dfs)):
    orig_consp = orig_consp.append(pd.read_csv(csv_path+'corona_parsed_orig_%d.csv'%i))  
    qtd_consp = qtd_consp.append(pd.read_csv(csv_path+'corona_parsed_qtd_%d.csv'%i))  
    rt_consp = rt_consp.append(pd.read_csv(csv_path+'corona_parsed_rt_%d.csv'%i))

/nas/home/jpatel/ls/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0,1,10,14,15,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [37]:
with open("../keys.json") as keys_json:
    keys = json.load(keys_json)

In [38]:
#Add your credentials here
twitter_keys = {
        'consumer_key':        keys['twitter_keys']['consumer_key'],
        'consumer_secret':     keys['twitter_keys']['consumer_secret'],
        'access_token_key':    keys['twitter_keys']['access_token_key'],
        'access_token_secret': keys['twitter_keys']['access_token_secret']
    }

auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True)

In [39]:
orig_consp.shape

(173989, 27)

In [82]:
# Format is each tweet is a json line in a .txt file
# {tweet_1_id : [retweet_jsons]}

group_size = 10000

save_file = "retweet_jsons"
for row in orig_consp.iterrows():
    if row[0] % group_size == 0:
        outfile = open(save_path + save_file + str(row[0]//group_size) + '.txt', 'w')
    t_id = row[1].tweetid
    try:
        rts = api.retweets(t_id, 100)
        this_list = []
        for rt in rts:
            this_list.append(rt._json)

        this_json = {t_id:this_list}
        json.dump(this_json, outfile)
        outfile.write('\n')
    except Exception as e:
        pass

KeyboardInterrupt: 

In [83]:
row

(708,
 tweetid                                                1222128545100488712
 userid                                                 1216621735643992064
 screen_name                                                hkpeoplebewate1
 date                                        tue jan 28 12:05:25 +0000 2020
 lang                                                                    en
 location                                                               NaN
 text                     people repulic of coronvirus( p.r.c.)  #prc #c...
 tweet_type                                                        original
 followers_count                                                          0
 favourites_count                                                        74
 verified                                                             False
 hashtag                  ['prc', 'china', 'antichinazi', 'coronavirus',...
 mentionid                                                               []
 menti